<a href="https://colab.research.google.com/github/kaicho8636/pyprover/blob/master/truth_value_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers

In [2]:
import pickle
from tqdm import tqdm

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data 

In [4]:
!git clone https://github.com/kaicho8636/pyprover
input_data = pickle_load('pyprover/formulus_data/input_data-3.pickle')
output_data = pickle_load('pyprover/formulus_data/output_data-3.pickle')

fatal: destination path 'pyprover' already exists and is not an empty directory.


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test_val, Y_train, Y_test_val = train_test_split(input_data, output_data,
                                                    test_size=0.1, random_state=42)

X_test, X_val = X_test_val[:len(X_test_val)//2], X_test_val[len(X_test_val)//2:]
Y_test, Y_val = Y_test_val[:len(Y_test_val)//2], Y_test_val[len(Y_test_val)//2:]

maxlen = 49
vocab_size = 9

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
from keras.callbacks import LearningRateScheduler
def decay_schedule(epoch, lr):
    if not epoch == 0:
        lr = lr * 0.95 
    return lr 

In [ ]:
def FCBlock(x, drop, lay_size, add_att):  
    x = layers.Dropout(drop)(x) 
    x = layers.Dense(lay_size)(x)
    if add_att != None:
        x = layers.Add()([x,add_att])
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    return x

def TransformerBlock(inputs, embed_dim, num_heads):
    att_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    return layers.BatchNormalization()(att_output + inputs)    

def get_model(num_heads=3, embed_dim=20, drop=0.1, lay_size=980, transformer=True):
    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size+1, embed_dim)
    x = embedding_layer(inputs)
    if transformer:
        x_shortcut_transformer = x

        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads)
        x = layers.Add()([x, x_shortcut_transformer])

        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads) 
        x = layers.Add()([x, x_shortcut_transformer])

        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads)
        x = TransformerBlock(x, embed_dim, num_heads) 
        x = layers.Add()([x, x_shortcut_transformer])             

    x = layers.Flatten()(x)

    x_shortcut = x
    x = FCBlock(x, drop, lay_size, None)    
    x = FCBlock(x, drop, lay_size, None)
    x = FCBlock(x, drop, lay_size, None)
    x = FCBlock(x, drop, lay_size, x_shortcut)

    x = FCBlock(x, drop, lay_size, None)    
    x = FCBlock(x, drop, lay_size, None)
    x = FCBlock(x, drop, lay_size, None)
    x = FCBlock(x, drop, lay_size, x_shortcut)        

    outputs = layers.Dense(8, activation="sigmoid")(x)

    adam = optimizers.Adam(lr=0.001)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=adam, loss="binary_crossentropy")
    lr_scheduler = LearningRateScheduler(decay_schedule)

    model.fit(
        X_train, Y_train, batch_size=32, epochs=10, callbacks=[lr_scheduler], validation_data=(X_val, Y_val)
    ) 
    predictions = model.predict(X_test[0:10])
    print(predictions)

get_model()    

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
2138/7836 [=======>......................] - ETA: 2:35 - loss: 0.4451

KeyboardInterrupt: ignored